This code-along analyses data from a survey about the growth of Finnish companies. The data reports the perceptions of top managers on growth, innovativeness, and the ability for renewal.

### Where is the data from?

- [Suominen & Pihlajamaa, 2022](https://www.sciencedirect.com/science/article/pii/S2352340922005261)
- [The dataset](https://zenodo.org/records/5820394#.Y5OKl-zMK3I)

### What will I learn today?

- How to summarize and visualize questions with a numeric response using a histogram.
- How to determine whether there is a difference between two groups of numeric responses using a Mann-Whitney U test.
- How to summarize and visualize questions with a categorical response using a bar plot.

In [9]:
import sys
import plotly.express as px
from scipy.stats import mannwhitneyu
from pathlib import Path #for relative path handling
import pandas as pd 
import duckdb


In [10]:
#Function to load any csv file by name thus avoiding chaning csv_path every time.
sys.path.append(str(Path().resolve().parent / "shared" / "utils"))

from data_loading import load_csv

# Robust, reusable CSV loading
df = load_csv('survey_data.csv')
if df is not None:
    display(df.head())

from data_loading import duckdb_read_csv

# Robust, reusable DuckDB CSV loading
duck_df = duckdb_read_csv('survey_data.csv')
if duck_df is not None:
    display(duck_df.head())


✅ CSV loaded successfully: c:\Users\mashel\Desktop\ml-scaffolds\ml-platform\data\raw\survey_data.csv


,,Growth_Firm;question_2_row_1_transformed;question_2_row_2_transformed;question_3_row_1;question_3_row_2;question_3_row_3;question_3_row_4;question_3_row_5;question_3_row_6;question_3_row_7;question_3_row_8;question_3_row_9;question_3_row_10;question_3_row_11;question_3_row_12;question_3_row_13;question_3_row_14;question_3_row_15;question_3_row_16;question_4_row_1;question_4_row_2;question_4_row_3;question_4_row_4;question_5_row_1;question_5_row_2;question_5_row_3;question_5_row_4;question_5_row_5;question_5_row_6;question_5_row_7;question_5_row_8;question_5_row_9;question_5_row_10;question_6_row_1;question_6_row_2;question_7_row_1
0;35,1351351351351;50,7509391319659;4;5;5;4;3;3;4;4;4;2;2;2;2;4;4;3;...
0;23,0180426462548;51,182200341316;5;4;4;4;4;4;4;5;5;4;2;4;2;4;4;3;4...
0;86,6404715127701;62,9326385264931;3;4;4;4;4;3;4;5;3;3;3;5;3;4;4;4;...
0;17,6470588235294;39,1304347826087;3;4;5;4;4;4;5;5;3;3;4;5;4;4;5;3;...
0;60;32,8021248339973;4;4;4;4;3;4;4;4;5;5;2;3;1;2;4;2;4;2;2;2;2;2;2;4;2;4;2;3;3;4;5;2;2,NaN


,Growth_Firm,question_2_row_1_transformed,question_2_row_2_transformed,question_3_row_1,question_3_row_2,question_3_row_3,question_3_row_4,question_3_row_5,question_3_row_6,question_3_row_7,...,question_5_row_4,question_5_row_5,question_5_row_6,question_5_row_7,question_5_row_8,question_5_row_9,question_5_row_10,question_6_row_1,question_6_row_2,question_7_row_1
0,0,35.135135,50.750939,4,5,5,4,3,3,4,...,4,2,4,2,3,2,5,4,5,1
1,0,23.018043,51.182200,5,4,4,4,4,4,4,...,3,4,3,3,3,4,3,5,4,1
2,0,86.640472,62.932639,3,4,4,4,4,3,4,...,5,4,4,4,4,<NA>,<NA>,5,3,1
3,0,17.647059,39.130435,3,4,5,4,4,4,5,...,3,3,4,4,4,3,3,3,3,1
4,0,60.000000,32.802125,4,4,4,4,3,4,4,...,4,2,4,2,3,3,4,5,2,2


In [11]:
#Create a duckdb connection for reuse
from data_loading import get_csv_path

csv_path = get_csv_path('survey_data.csv')
count = duckdb.query(f"""
    SELECT COUNT(*) FROM read_csv_auto(
        '{csv_path.as_posix()}',
        delim=';',
        decimal_separator=',',
        nullstr=' '
    )
""").fetchone()[0]
print(f"Row count: {count}")
    
    

Row count: 120


In [4]:
# Select everything from survey_data.csv

# Query survey_data.csv (adjust filename if needed)
query = f"""
SELECT * FROM read_csv_auto(
    '{csv_path.as_posix()}',
    delim=';',
    decimal_separator=',',
    nullstr=' '
)
"""
df = duckdb.query(query).to_df()
df.head()

,Growth_Firm,question_2_row_1_transformed,question_2_row_2_transformed,question_3_row_1,question_3_row_2,question_3_row_3,question_3_row_4,question_3_row_5,question_3_row_6,question_3_row_7,...,question_5_row_4,question_5_row_5,question_5_row_6,question_5_row_7,question_5_row_8,question_5_row_9,question_5_row_10,question_6_row_1,question_6_row_2,question_7_row_1
0,0,35.135135,50.750939,4,5,5,4,3,3,4,...,4,2,4,2,3,2,5,4,5,1
1,0,23.018043,51.182200,5,4,4,4,4,4,4,...,3,4,3,3,3,4,3,5,4,1
2,0,86.640472,62.932639,3,4,4,4,4,3,4,...,5,4,4,4,4,<NA>,<NA>,5,3,1
3,0,17.647059,39.130435,3,4,5,4,4,4,5,...,3,3,4,4,4,3,3,3,3,1
4,0,60.000000,32.802125,4,4,4,4,3,4,4,...,4,2,4,2,3,3,4,5,2,2


In [5]:
# Select everything from survey_questions.csv

csv_path = get_csv_path('survey_questions.csv')
query = f"""
SELECT * FROM read_csv_auto(
    '{csv_path.as_posix()}',
    delim=';',
    decimal_separator=',',
    nullstr=' '
)
"""
df_questions = duckdb.query(query).to_df()
df_questions.head()

,"column,question,row,section,title,response_type"
0,"question_2_row_1_transformed,2,1,estimated gro..."
1,"question_2_row_2_transformed,2,2,estimated gro..."
2,"question_3_row_1,3,1,company culture,Employees..."
3,"question_3_row_2,3,2,company culture,Managers ..."
4,"question_3_row_3,3,3,company culture,Employees..."


In [6]:
#Visualize with histogram
px.histogram(
  duck_df,
  x="question_2_row_1_transformed",
  labels={
        "question_2_row_1_transformed": "Expected employee count in five years (as a percent from last available year)"
    }
)

In [7]:
px.histogram(
    duck_df, 
    x="question_2_row_1_transformed",
    facet_row="Growth_Firm",
    labels={
        "question_2_row_1_transformed": "Expected employee count in five years (as a percent from last available year)"
    }
)